4 - Topic Modelling

Techniques to be used: LDA (Latent Dirichlet Allocation)
    
Input: Docterm matrix

In [1]:
# take the dtm.pkl and load to dataframe
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop1.pkl')
len(data)

10

In [2]:
from gensim import matutils, models
import scipy.sparse

In [3]:
tdm = data.transpose()
tdm.head()

,Deepak Ravishankar Ramkumar,Karthik Kottiswaran,Karthikeyan Narayanaswami,Krishna Kumar,Raj Kumar,Raj Kumar S,Ram,Ram Balaji Subbaiyan,Riyas Khan,Selva Vignesh
aache,1,0,0,0,0,0,0,0,0,0
aaga,0,1,0,0,0,0,0,0,0,0
aagura,1,0,0,0,0,0,0,0,0,0
aaguthu,0,0,0,1,0,0,0,0,0,0
aahirum,0,0,0,0,0,0,0,0,1,0


In [4]:
# put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [5]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("data_clean2.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [6]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.005*"time" + 0.005*"image" + 0.004*"gif" + 0.004*"nu" + 0.004*"oru" + 0.004*"group" + 0.004*"iruka" + 0.003*"think" + 0.003*"vijay" + 0.003*"ellam"'),
 (1,
  '0.009*"krishna" + 0.009*"ram" + 0.008*"lol" + 0.007*"nu" + 0.007*"deepak" + 0.006*"murthy" + 0.005*"karthik" + 0.004*"image" + 0.004*"nee" + 0.004*"selva"')]

In [7]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.006*"ellam" + 0.005*"time" + 0.005*"gif" + 0.005*"dude" + 0.005*"athu" + 0.004*"nu" + 0.004*"iruka" + 0.004*"job" + 0.004*"try" + 0.003*"image"'),
 (1,
  '0.014*"krishna" + 0.012*"lol" + 0.012*"ram" + 0.011*"nu" + 0.008*"murthy" + 0.008*"nee" + 0.007*"deepak" + 0.005*"riyas" + 0.005*"selva" + 0.004*"time"'),
 (2,
  '0.006*"image" + 0.006*"think" + 0.005*"like" + 0.005*"time" + 0.005*"did" + 0.004*"know" + 0.004*"dont" + 0.003*"best" + 0.003*"group" + 0.003*"karthik"')]

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). 
Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [8]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [9]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean1.pkl')
data_clean

,transcript
Deepak Ravishankar Ramkumar,right up your alley macha can convince inter...
Karthik Kottiswaran,watha unga kozha adi sandaila yen thaliya aruk...
Karthikeyan Narayanaswami,do you really have that much money un per...
Krishna Kumar,unga english padichu meaning kan...
Raj Kumar,this happens there are companies that t even ...
Raj Kumar S,u wont be in trouble r the trouble gif i can ...
Ram,sticker i saw something like yesterday a jd ...
Ram Balaji Subbaiyan,image enoda maximum eh yr months in ...
Riyas Khan,sticker sticker yaaru master ellah sari a...
Selva Vignesh,wat yenne pannen even i remember this paith...


In [10]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
Deepak Ravishankar Ramkumar,alley macha years experience change relationsh...
Karthik Kottiswaran,watha kozha adi sandaila yen thaliya years peo...
Karthikeyan Narayanaswami,money tha varuthu i account tamil movies zee i...
Krishna Kumar,padichu time aaguthu months week kanaku solran...
Raj Kumar,companies profile companies profile experience...
Raj Kumar S,wont trouble r trouble gif i reaction place wa...
Ram,sticker i something yesterday jd years experie...
Ram Balaji Subbaiyan,image enoda months job dude fillings details e...
Riyas Khan,sticker sticker yaaru master ellah sari aahiru...
Selva Vignesh,wat yenne paithiyam thaniya irukule dance clas...


In [13]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']

stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,aache,aaga,aagura,aaguthu,aahirum,aaiduvan,aal,aalu,aana,abi,...,yenne,yeppadi,yes,yesterday,yevlo,yjmkyefcpl,yosichu,yosikama,youtu,zee
Deepak Ravishankar Ramkumar,1,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Karthik Kottiswaran,0,1,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
Karthikeyan Narayanaswami,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Krishna Kumar,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
Raj Kumar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Raj Kumar S,0,0,0,0,0,0,0,1,0,3,...,0,0,0,0,0,0,0,0,1,0
Ram,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
Ram Balaji Subbaiyan,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
Riyas Khan,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Selva Vignesh,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [14]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [15]:
# Let's start with 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.011*"image" + 0.010*"gif" + 0.007*"group" + 0.006*"job" + 0.006*"dude" + 0.006*"kid" + 0.006*"ellam" + 0.004*"https" + 0.004*"things" + 0.004*"money"'),
 (1,
  '0.025*"krishna" + 0.025*"ram" + 0.015*"murthy" + 0.014*"deepak" + 0.010*"lol" + 0.010*"selva" + 0.009*"riyas" + 0.008*"image" + 0.008*"karthik" + 0.008*"balaji"'),
 (2,
  '0.007*"companies" + 0.007*"life" + 0.007*"watha" + 0.005*"day" + 0.005*"group" + 0.005*"skills" + 0.005*"vijay" + 0.005*"profile" + 0.005*"years" + 0.005*"person"'),
 (3,
  '0.008*"deepak" + 0.008*"image" + 0.008*"karthik" + 0.007*"sticker" + 0.007*"machan" + 0.007*"week" + 0.006*"ram" + 0.006*"years" + 0.006*"raj" + 0.005*"brother"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives

In [16]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [17]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
Deepak Ravishankar Ramkumar,alley macha years total experience change rela...
Karthik Kottiswaran,watha unga kozha adi sandaila yen thaliya many...
Karthikeyan Narayanaswami,much money un tha varuthu i account tamil movi...
Krishna Kumar,unga english padichu kandupudikrathuke time aa...
Raj Kumar,companies profile more companies entire profil...
Raj Kumar S,u wont trouble r trouble gif i ohoh reaction p...
Ram,sticker i something yesterday jd years experie...
Ram Balaji Subbaiyan,image enoda maximum eh yr months first job the...
Riyas Khan,sticker sticker yaaru master ellah sari aahiru...
Selva Vignesh,wat yenne i paithiyam thaniya irukule dance cl...


In [18]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aache,aaga,aagura,aaguthu,aahirum,aaiduvan,aal,aalu,aana,aanmagan,...,yesterday,yethukalaam,yevlo,yjmkyefcpl,yosichu,yosikama,youngest,youtu,yr,zee
Deepak Ravishankar Ramkumar,1,0,1,0,0,0,2,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Karthik Kottiswaran,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Karthikeyan Narayanaswami,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Krishna Kumar,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Raj Kumar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Raj Kumar S,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
Ram,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
Ram Balaji Subbaiyan,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,1,0,0,1,0
Riyas Khan,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Selva Vignesh,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [19]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [20]:
# Let's start with 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"image" + 0.006*"dude" + 0.005*"cisco" + 0.005*"years" + 0.005*"job" + 0.004*"deepak" + 0.004*"karthik" + 0.004*"good" + 0.004*"ellam" + 0.004*"week"'),
 (1,
  '0.008*"krishna" + 0.006*"lol" + 0.006*"companies" + 0.006*"raj" + 0.006*"adhuku" + 0.004*"nu" + 0.004*"nee" + 0.004*"riyas" + 0.004*"skills" + 0.004*"profile"'),
 (2,
  '0.001*"image" + 0.001*"nee" + 0.001*"nu" + 0.001*"ram" + 0.001*"iruka" + 0.001*"oru" + 0.001*"gif" + 0.001*"group" + 0.001*"deepak" + 0.001*"krishna"'),
 (3,
  '0.016*"ram" + 0.013*"krishna" + 0.011*"nu" + 0.011*"murthy" + 0.009*"image" + 0.008*"lol" + 0.008*"deepak" + 0.007*"thambi" + 0.007*"nee" + 0.007*"selva"')]

## Identify Topics in Each Document
Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [22]:
#  Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.009*"image" + 0.005*"years" + 0.005*"karthik" + 0.005*"group" + 0.005*"th" + 0.005*"kid" + 0.005*"deepak" + 0.004*"true" + 0.004*"week" + 0.004*"best"'),
 (1,
  '0.007*"nu" + 0.005*"ellam" + 0.005*"ram" + 0.004*"better" + 0.004*"raj" + 0.004*"iruka" + 0.004*"dude" + 0.004*"athu" + 0.004*"group" + 0.004*"nee"'),
 (2,
  '0.001*"free" + 0.001*"zee" + 0.001*"tv" + 0.001*"adei" + 0.001*"devices" + 0.001*"tha" + 0.001*"tamil" + 0.001*"account" + 0.001*"hindi" + 0.001*"live"'),
 (3,
  '0.021*"ram" + 0.019*"krishna" + 0.015*"murthy" + 0.013*"nu" + 0.013*"deepak" + 0.011*"lol" + 0.010*"nee" + 0.009*"selva" + 0.007*"riyas" + 0.007*"thambi"')]

In [23]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(0, 'Deepak Ravishankar Ramkumar'),
 (1, 'Karthik Kottiswaran'),
 (0, 'Karthikeyan Narayanaswami'),
 (1, 'Krishna Kumar'),
 (3, 'Raj Kumar'),
 (3, 'Raj Kumar S'),
 (0, 'Ram'),
 (1, 'Ram Balaji Subbaiyan'),
 (3, 'Riyas Khan'),
 (1, 'Selva Vignesh')]